In [1]:
import csv
import numpy as np
# TUH_index = np.chararray()
with open('_SEIZURES_v28r_eval.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    x = list(readCSV)
    x = np.array(x).astype('str')
    x = x[2:1423,:]
    print(x)
    print(x.shape)

[['1' '1' '258' ..., '256' '40%' '40%']
 ['2' '2' '' ..., '186' '29%' '68%']
 ['3' '3' '258' ..., '6' '1%' '69%']
 ..., 
 ['1419' '1030' '6546' ..., '' '' '']
 ['1420' '1031' '6546' ..., '' '' '']
 ['1421' '1032' '' ..., '' '' '']]
(1421, 30)


In [2]:
preictal_advance = 10
seizure_intervals = []
lead_seizure_intervals = []
lead_seizure_file_idx = []
prev_file_idx = 0
for i in range(x.shape[0]):
    cur_idx = x[i,0]
    cur_file_idx = x[i,1]
    if x[i,12] != '':
        seizure_intervals.append((x[i,12],x[i,13]))    
#         if (float(x[i,13]) - float(x[i,12])) > 2000:
#             print((i, x[i,12],x[i,13]))
    if cur_file_idx != prev_file_idx:
        if x[i,12] != '':
            if float(x[i,12]) >= preictal_advance+1:
                lead_seizure_intervals.append((x[i,12],x[i,13]))
                lead_seizure_file_idx.append(i)
    prev_idx = cur_idx
    prev_file_idx = cur_file_idx
print('number of seizures: ',len(seizure_intervals))
print('number of lead seizures: ',len(lead_seizure_intervals))
print('number of lead seizures with preictal period > 60s: ',len([float(i[0]) for i in lead_seizure_intervals if float(i[0]) > 60]))
print('total preictal period > 60s: ',sum([float(i[0]) for i in lead_seizure_intervals if float(i[0]) > 60])/60)
seizure_durations = [(float(i[1]) - float(i[0])) for i in seizure_intervals]
print('total duration of seizures in min: ', sum(seizure_durations)/60)
print('minimum duration of seizures: ', min(seizure_durations))
print('maximum duration of seizures: ', max(seizure_durations))
seizure_start_times = [float(i[0]) for i in seizure_intervals]

number of seizures:  646
number of lead seizures:  217
number of lead seizures with preictal period > 60s:  176
total preictal period > 60s:  849.5573699999999
total duration of seizures in min:  929.415573333333
minimum duration of seizures:  1.3999999999999773
maximum duration of seizures:  2464.0


In [3]:
preictal_file_name = []
preictal_start_times = []
for i in range(len(lead_seizure_file_idx)):
    idx = lead_seizure_file_idx[i]
    preictal_file_name.append(x[idx, 11])
    preictal_start_times.append(float(x[idx, 12]))
print(len(preictal_file_name))
nvalid_preictal_files = len(preictal_file_name)

217


In [4]:
import pyedflib as edf
import numpy as np
def find_EEG_channels(x):
    channels = []
    standard = ['FP1','FP2','F7','F3','FZ','F4','F8','A1','T3','C3','CZ','C4','T4','A2','T5','P3','PZ','P4','T6','O1','O2']
    #standard = ['FP1','FP2','F7','F3','FZ','F4','F8','A1','T3','C3','CZ','C4','T4','A2','T5','P3','PZ','P4','T6','O1','O2','EKG','SP1','SP2','31','32','LUC','RLC','RESP1','RESP2']
    for i in standard:
        for j in x:
            if 'EEG' in j:
                if i == j.partition('EEG ')[-1].rpartition('-')[0]:
                    channels.append(i)
    return channels
for i in range(len(preictal_file_name)):
    filename = preictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    cur_channels = find_EEG_channels(f.getSignalLabels())
    if i == 0:
        prev_channels = cur_channels
    common_channels = list(set(prev_channels) & set(cur_channels))
    #print(i,len(common_channels))
    prev_channels = common_channels
print(common_channels, len(common_channels))
print(set(common_channels) == set(['FP1','FP2','F7','F3','FZ','F4','F8','T3','C3','CZ','C4','T4','T5','P3','PZ','P4','T6','O1','O2']))
common_channels = ['FP1','FP2','F7','F3','FZ','F4','F8','T3','C3','CZ','C4','T4','T5','P3','PZ','P4','T6','O1','O2']


['CZ', 'C4', 'T6', 'F4', 'P4', 'C3', 'P3', 'T5', 'FP1', 'T3', 'FP2', 'T4', 'F7', 'PZ', 'O2', 'FZ', 'O1', 'F8', 'F3'] 19
True


In [5]:
nvalid_channels = len(common_channels)
channel_idx_per_preictal = np.zeros((nvalid_preictal_files, nvalid_channels))
for i in range(len(preictal_file_name)):
    filename = preictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    valid_channels = []
    valid_channel_names = []
    for channel_idx, channel in enumerate(common_channels):
        for sample_channel_idx in range(len(f.getSignalLabels())):
            sample_channel_name = f.getLabel(sample_channel_idx)
#             if 'EEG' in sample_channel_name:
#                 print(sample_channel_name.partition('EEG ')[-1].rpartition('-')[0])
            if 'EEG' in sample_channel_name:
                if sample_channel_name.partition('EEG ')[-1].rpartition('-')[0] == channel:
                    channel_idx_per_preictal[i,channel_idx] = sample_channel_idx
                    valid_channels.append(sample_channel_idx)
                    valid_channel_names.append(sample_channel_name)

In [6]:
from scipy.signal import resample
target_f = 200
def down_sample(x, samp_freq, target_freq):
    x = np.array(x)
    original_len = x.shape[0]
    target_len = int(original_len*target_freq/samp_freq)
    target_x= resample(x, target_len)
    return target_x
for i in range(len(preictal_file_name)):
    filename = preictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    samp_freq = f.getSampleFrequencies()[0]
    nsecs = np.floor(preictal_start_times[i]-preictal_advance)
    #print(nsecs,samp_freq)
    data = np.zeros((nvalid_channels, int(nsecs*target_f)))
    processed_file_name = 'Processed_data_10s//eval//TUH_preictal_eval_{0}'.format(str(i+1).zfill(3))
    for j in range(nvalid_channels):
        #print(i,j)
        c = channel_idx_per_preictal[i,j]
        #print(f.readSignal(int(c), start = 0))
        try:
            data[j,:] = down_sample(f.readSignal(int(c))[0:int(nsecs*f.getSampleFrequency(int(c)))], f.getSampleFrequency(int(c)), target_f)
        except:
            print(i,j)
    np.save(processed_file_name, data)    

In [7]:
inter_seizure_file_idx = []
for i in range(x.shape[0]):
    cur_idx = x[i,0]
    cur_file_idx = x[i,1]
    if x[i,12] == '':
        inter_seizure_file_idx.append(i)    
print('number of interictal sessions: ',len(inter_seizure_file_idx))
interictal_file_name = []
for i in range(len(inter_seizure_file_idx)):
    idx = inter_seizure_file_idx[i]
    interictal_file_name.append(x[idx, 11])
print(len(interictal_file_name))
ninterictal_files = len(interictal_file_name)

number of interictal sessions:  775
775


In [8]:
for i in range(ninterictal_files):
    filename = interictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    cur_channels = find_EEG_channels(f.getSignalLabels())
    if i == 0:
        prev_channels = cur_channels
    common_channels_interictal = list(set(prev_channels) & set(cur_channels))
    #print(i,len(common_channels))
    prev_channels = common_channels_interictal
print(common_channels_interictal, len(common_channels_interictal))

['O1', 'T4', 'F7', 'T5', 'T3', 'P3', 'P4', 'T6', 'O2', 'CZ', 'F3', 'FP2', 'F8', 'C4', 'F4', 'FP1', 'PZ', 'C3', 'FZ'] 19


In [8]:
channel_idx_per_interictal = np.zeros((ninterictal_files, nvalid_channels))
for i in range(ninterictal_files):
    filename = interictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    for channel_idx, channel in enumerate(common_channels):
        for sample_channel_idx in range(len(f.getSignalLabels())):
            sample_channel_name = f.getLabel(sample_channel_idx)
#             if 'EEG' in sample_channel_name:
#                 print(sample_channel_name.partition('EEG ')[-1].rpartition('-')[0])
            if 'EEG' in sample_channel_name:
                if sample_channel_name.partition('EEG ')[-1].rpartition('-')[0] == channel:
                    channel_idx_per_interictal[i,channel_idx] = sample_channel_idx

for i in range(ninterictal_files):
    filename = interictal_file_name[i]
    f = edf.EdfReader(filename.replace('tse' , 'edf'))
    samp_freq = f.getSampleFrequencies()[0]
    nsecs = np.floor(f.getFileDuration())
    data = np.zeros((nvalid_channels, int(nsecs*target_f)))
    processed_file_name = 'Processed_data_10s//eval//TUH_interictal_eval_{0}'.format(str(i+1).zfill(3))
    for j in range(nvalid_channels):
        c = channel_idx_per_interictal[i,j]
        #print(f.readSignal(int(c), start = 0))
        data[j,:] = down_sample(f.readSignal(int(c))[0:int(nsecs*f.getSampleFrequency(int(c)))], f.getSampleFrequency(int(c)), target_f)
    np.save(processed_file_name, data)    